In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd
import os
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares


os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 2) 분석해 봅시다. 

In [7]:
# ratings에 있는 유니크한 영화 개수
print("유니크한 영화 개수 : ",ratings['movie_id'].nunique(),"개")

유니크한 영화 개수 :  3628 개


In [8]:
#rating에 있는 유니크한 사용자 수
print("유니크한 사용자 수 : ", ratings['user_id'].nunique(),"개")

유니크한 사용자 수 :  6039 개


In [9]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)

In [10]:
data.head()

,movie_id,title,genre,user_id,counts,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [11]:
# 사용하는 것만 남겨줍니다. 

data=data[[ 'user_id','title', 'counts']]
data.head()

,user_id,title,counts
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5


In [12]:
data['title'] = data['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,title,counts
0,1,toy story (1995),5
1,6,toy story (1995),4
2,8,toy story (1995),4
3,9,toy story (1995),5
4,10,toy story (1995),5
5,18,toy story (1995),4
6,19,toy story (1995),5
7,21,toy story (1995),3
8,23,toy story (1995),4
9,26,toy story (1995),3


### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [13]:
#가장 인기있는 영화 30개(인기순)
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [14]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['men in black (1997)' , 'ghostbusters (1984)' ,'terminator, the (1984)' ,
               'back to the future (1985)' ,'american beauty (1999)']

# 'youngrae'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ["youngrae"]*5, 'title': my_favorite, 'counts':[5]*5})

# user_id에 'youngrae'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['youngrae']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,counts
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


### 모델에 활용하기 위한 전처리

In [15]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [17]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['youngrae'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['american beauty (1999)'])

6039
2600


In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1419,5
1,6039,2462,5
2,6039,1122,5
3,6039,1152,5


### 4) CSR matrix를 직접 만들어 봅시다.

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [21]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [25]:
youngrae, back_to_the_future = user_to_idx['youngrae'], title_to_idx['back to the future (1985)']
youngrae_vector, back_to_the_future_vector = als_model.user_factors[youngrae], als_model.item_factors[back_to_the_future]

In [27]:
youngrae_vector

array([ 0.06251419,  0.26313746, -0.7619017 ,  0.09420431, -0.29129565,
        0.24524154,  0.23299329,  0.10571272, -0.06843271, -0.11987498,
        0.9259833 , -0.08598096,  0.71617275,  0.06149281, -0.2773859 ,
        0.32643953,  0.5232237 , -0.21105523, -0.34421498, -0.9200823 ,
        0.6247948 , -0.20649801, -0.31577587, -0.10954741, -0.3838594 ,
       -0.00146131,  0.10753657,  0.22224417,  0.06989548, -0.04872292,
       -0.8725765 , -0.00672697, -0.30904713, -0.5417306 ,  0.09575019,
       -0.3910624 , -0.10013445,  0.5848849 ,  0.513809  ,  0.91751325,
       -0.85899127, -0.34144807, -0.34988418,  0.0888447 , -0.32051125,
        0.76033396, -0.5287318 ,  0.1277109 ,  0.1927637 ,  0.16877979,
       -0.59720695, -0.19928396, -0.38875327,  0.39281067,  0.33947003,
        0.45374864,  0.860692  ,  0.90161824, -0.5809986 , -0.5427231 ,
        0.66091377,  0.687624  , -0.00299371, -0.4268446 ,  0.6614803 ,
       -0.5187278 ,  1.0937243 , -0.39006615,  0.61995673,  0.20

In [28]:
back_to_the_future_vector

array([ 0.00172642,  0.02168334, -0.0012364 ,  0.00861844,  0.00401814,
       -0.015096  ,  0.00939871,  0.01911605,  0.00317465,  0.00769319,
        0.06569176, -0.00147625,  0.03947211, -0.00671237, -0.00100433,
        0.00255486,  0.00790469,  0.00603146,  0.01409669, -0.01997965,
        0.02244654,  0.03435861, -0.01858568, -0.00733934, -0.02858197,
        0.01543553, -0.0182959 ,  0.02022886,  0.00066868,  0.01353501,
       -0.01121133, -0.0101115 ,  0.01287295, -0.01564965, -0.00573527,
       -0.01600438, -0.0084985 ,  0.02210689,  0.02236356,  0.03700649,
       -0.01886008, -0.0082798 ,  0.00776734,  0.01300444,  0.00154893,
        0.05175604,  0.0238166 , -0.00560182,  0.0248694 ,  0.02929958,
       -0.01507108, -0.01539772, -0.00896776,  0.01815256,  0.02808357,
        0.0309196 ,  0.00828212,  0.01677601, -0.02334757, -0.01632266,
        0.02910498,  0.01861904,  0.00844146, -0.00263666,  0.0067912 ,
       -0.0072588 ,  0.04860748, -0.01600755,  0.01994748,  0.01

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


In [30]:
np.dot(youngrae_vector, back_to_the_future_vector)

0.64261633

나의 back_to_the_future 영화의 선호도는 64% 가 나왔다. 그렇다면 터미네이터는 어떻게 나올까?

In [35]:
Terminator=title_to_idx['terminator, the (1984)']
Terminator_vector = als_model.item_factors[Terminator]
np.dot(youngrae_vector, Terminator_vector)

0.36360353

선호하는 영화가 그래도 더 높게 나온 것을 확인할 수 있었다.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [37]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [38]:
get_similar_title('men in black (1997)') 

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'matrix, the (1999)',
 'independence day (id4) (1996)',
 'lost world: jurassic park, the (1997)',
 'fifth element, the (1997)',
 'face/off (1997)',
 'schlafes bruder (brother of sleep) (1995)']

In [39]:
get_similar_title('terminator 2: judgment day (1991)') 

['terminator 2: judgment day (1991)',
 'matrix, the (1999)',
 'jurassic park (1993)',
 'total recall (1990)',
 'terminator, the (1984)',
 'men in black (1997)',
 'fugitive, the (1993)',
 'braveheart (1995)',
 'schlafes bruder (brother of sleep) (1995)',
 'hunt for red october, the (1990)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [40]:
user = user_to_idx['youngrae']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

jurassic park (1993) 0000.528
terminator 2: judgment day (1991) 0000.500
star wars: episode vi - return of the jedi (1983) 0000.404
star wars: episode iv - a new hope (1977) 0000.390
matrix, the (1999) 0000.383
star wars: episode v - the empire strikes back (1980) 0000.368
e.t. the extra-terrestrial (1982) 0000.360
being john malkovich (1999) 0000.359
sixth sense, the (1999) 0000.358
airplane! (1980) 0000.356
galaxy quest (1999) 0000.326
braveheart (1995) 0000.281
shakespeare in love (1998) 0000.273
princess bride, the (1987) 0000.268
total recall (1990) 0000.263
stand by me (1986) 0000.240
raiders of the lost ark (1981) 0000.235
star wars: episode i - the phantom menace (1999) 0000.226
abyss, the (1989) 0000.222
alien (1979) 0000.221


In [48]:
jurassic_park = title_to_idx['jurassic park (1993)']
explain = als_model.explain(user, csr_data, itemid=jurassic_park)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('men in black (1997)', 0.3391942910339937),
 ('back to the future (1985)', 0.07857422177081785),
 ('american beauty (1999)', 0.05921048252328726),
 ('terminator, the (1984)', 0.04437239212631578),
 ('ghostbusters (1984)', -0.0005506273081347442)]

쥬라기 공원을 넣었을때 다음과 같은 영화들을 추천 받았다. 

###  회고

이번 프로젝트는 추천 시스템에 대한 내용이었다.
사용자가 선호하는 항목을 토대로 유사한 항목을 추천하는 시스템이다.
iterations를 높일 수록 내가 선택한 영화와 행렬곱을 한 결과가 높아졌다.
높은 정확도가 나오면 좋을 것 같지만, 내가 선호하는 다른 영화의 정확도는 떨어지는 단점이 있다.
이 두가지를 적절하게 조절하는 시스템이 좋은 추천 시스템인듯 하다.
조금 아쉬웠던 점은 데이터 상에 있는 영화들이라 잘 알지 못하는 영화 5가지를 선택해서 사실 추천받은 영화들도 정말로 나한테 맞는건지 아니면 그저 유명해서 추천받은건지 잘 모르겠다.
그래도 모델이 잘 작동한 것 같아 기분은 좋다. 